### Churn @ Robinhood
#### Load Libraries and Data

In [ ]:
from numba import cuda 
print(cuda.detect())

In [ ]:
import cudf as cf
equity_df_raw = cf.read_csv('./data/equity_value_data.csv')
features_df_raw = cf.read_csv('./data/features_data.csv')

equity_df = equity_df_raw.copy()
features_df = features_df_raw.copy()

In [ ]:
equity_df.info()

In [ ]:
features_df.info()

#### a). What percentage of users have churned in the data?
A user is *churned* when their equity falls below 10 usd for 28 consecutive calendar days or longer having perviously been at least 10 usd

In [ ]:
# to ensure timestamp is datetime
equity_df['timestamp'] = cf.to_datetime(equity_df['timestamp']).copy() 

# Sort Values by user_id and timestamp
equity_df = equitydf.sort_values(['user_id', 'timestamp']).copy()

# Flag close_equity below 10 dollars
equity_df['below_10'] = (equity_df['close_equity'] < 10).astype(int).copy()

# Compute rolling 28-day windows for each user
equity_df['below_10_28d'] = equity_df.groupby('user_id')['below_10'].rolling(window=28, min_periods=28).sum().reset_index(0,drop=True).copy()

# Identify churn accounts
equity_df['churn'] = (equity_df['below_10_28d'] == 28).astype(int).copy()

# Ensure customer previously had >= 10 dollars
equity_df['above_10_before'] = equity_df.groupby('user_id')['close_equity'].transform(lambda x: (x>= 10).any()).copy()

# Filter churned customers
churned_users = equity_df.loc[(equity_df['churn'] == 1) & (equity_df['above_10_before']), 'user_id'].uniqe().copy()

# Calculate churn percentage
total_users = equity_df['user_id'].nunique()
churned_percentage = (len(churned_users) / total_users) * 100

print(f'Percent churned: {churned_percentage:.2f}%')